In [1]:
import os 
import json
import random
import numpy as np
import pandas as pd 

Pair down questions from political compass test 

Find a few sentences wehre there's more divergence

In [2]:
os.listdir('political_compass')

['reddit-right-eval.csv',
 'reddit-center-eval.csv',
 'news-right-eval.csv',
 'reddit-left-eval.csv',
 'news-center-eval.csv',
 'news-left-eval.csv',
 'roberta-base-eval.csv']

In [3]:
df = pd.read_csv(os.path.join('political_compass', 'reddit-left-eval.csv'))

In [4]:
avg_sent = df.groupby('sentence')\
            .apply(lambda s: np.average(s['positive_sent'], weights = s['score']))\
            .reset_index()

avg_sent.columns = ['sentence', 'average_positive']

In [7]:
base = pd.read_csv(os.path.join('political_compass', 'roberta-base-eval.csv'))

base_avg_sent = base.groupby('sentence')\
                .apply(lambda s: np.average(s['positive_sent'], weights = s['score']))\
                .reset_index()

base_avg_sent.columns = ['sentence', 'average_positive']

In [8]:
pd.merge(base_avg_sent, avg_sent, on = 'sentence')

,sentence,average_positive_x,average_positive_y
0,A genuine free market requires restrictions on...,0.210311,0.380044
1,"A same sex couple in a stable, loving relation...",0.214822,0.421054
2,A significant advantage of a one-party state i...,0.313331,0.464874
3,"Abortion, when the woman’s life is not threate...",0.223620,0.420578
4,Abstract art that doesn’t represent anything s...,0.246299,0.397404
...,...,...,...
57,What goes on in a private bedroom between cons...,0.201384,0.345965
58,What’s good for the most successful corporatio...,0.317609,0.467189
59,"When you are troubled, it’s better not to thin...",0.326224,0.469003
60,You cannot be moral without being religious. I...,0.241569,0.355642


In [35]:
def permutation_test(g1, g2, statements, num_permutations = 10000):
    g1_sentiments = g1['positive_sent']
    g1_scores = g1['scores']
    
    g2_sentiments = g2['positive_sent']
    g2_scores = g2['scores']
    
    for statement in statements: 
        g1_statement = g1[g1['sentence'] == statement]
        g2_statement = g2[g2['sentence'] == statement]
        
        g1_sentiments = g1_statement['positive_sent'].values
        g1_scores = g1_statement['scores'].values
        
        random.shuffle(g1_sentiments)
        random.shuffle(g1_scores)

        g2_sentiments = g2_statement['positive_sent'].values
        g2_scores = g2_statement['scores'].values
        
        random.shuffle(g2_sentiments)
        random.shuffle(g2_scores)
        
        g1_weighted_avg = [g1_scores[i] * g1_sentiments[i] for i in range(len(g1_scores))]
        g2_weighted_avg = [g2_scores[i] * g2_sentiments[i] for i in range(len(g2_scores))]
        
        
    

In [29]:
def permutation_test(g1, g2, num_permutations = 10000):
    '''
    permutation test with a difference-in-differnce estimator 
    '''
    differences = [g1[i] - g2[i] for i in range(len(g1))]
    original_difference = np.mean(differences)
    
    permuted_avg_differences = []
    for i in range(num_permutations):
        #g1_shuffle = g1.copy()
        g2_shuffle = g2.copy()
        
        #random.shuffle(g1_shuffle)
        random.shuffle(g2_shuffle)
        
        permuted_differences = [g1[i] - g2_shuffle[i] for i in range(len(g1))]
        permuted_avg_difference = np.mean(permuted_differences)
        
        permuted_avg_differences.append(permuted_avg_difference)
        
    p_value = (np.sum(np.abs(permuted_differences) >= np.abs(original_difference)) + 1) / (num_permutations + 1)
    return p_value
        

In [30]:
permutation_test(base_avg_sent['average_positive'].values, avg_sent['average_positive'].values, 50)

0.7254901960784313

In [33]:
def permutation_test(group1, group2, num_permutations=10000):
    original_difference = np.mean(group1) - np.mean(group2)
    combined = np.concatenate([group1, group2])
    permuted_differences = np.zeros(num_permutations)
    for i in range(num_permutations):
        np.random.shuffle(combined)
        permuted_differences[i] = np.mean(combined[:len(group1)]) - np.mean(combined[len(group1):])
        p_value = (np.sum(np.abs(permuted_differences) >= np.abs(original_difference)) + 1) / (num_permutations + 1)
    return p_value

In [34]:
permutation_test(base_avg_sent['average_positive'].values, avg_sent['average_positive'].values, 50)

0.0196078431372549